## Document Pre-Processing
### 1. Load & Parse  
  - PyMuPDF4LLM (On-Premise, CPU-based, Baseline)  
  - Upstage Document AI (SaaS)  
  - LlamaParse (SaaS)  
  - Docling (OpenSource, VLM-based)  
  - PaddleOCR (OpenSource, VLM-based)

# 라이브러리 설치

- **LangChain-LangGraph** (≥1.0.0): langchain, langgraph, langchain-core, langchain-openai, langchain-text-splitters, langchain-community. 
- **Document Parsing**: 
  - pymupdf4llm (https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/)
  - docling (https://docling-project.github.io/docling/installation/)
  - paddleocr (https://www.paddlepaddle.org.cn/en/install/quick)
  - Upstage Document Parser(https://console.upstage.ai/docs/getting-started?api=document-parsing)
  - Llama Parse(https://developers.llamaindex.ai/python/cloud/llamaparse/getting_started/)
- **Semantic Chunking**: chonkie (https://github.com/chonkie-inc/chonkie)

## 버전 요구사항

- Python = 3.13
- LangChain ≥ 1.0.0
- LangGraph ≥ 1.0.0

In [ ]:
# ============================================================================
# 환경 확인 및 패키지 설치
# ============================================================================
import sys

# Python 버전 확인
python_version = sys.version_info
print(f"Python 버전: {python_version.major}.{python_version.minor}.{python_version.micro}")

if python_version < (3, 10):
    raise RuntimeError("❌ Python 3.10 이상 필요합니다. 현재 버전이 지원되지 않습니다.")
else:
    print(" Python 버전 OK")

# 핵심 패키지 설치
# 분할 설치로 dependency resolution 문제 회피
print("LangChain V1.0 설치 중...")
%pip install -qU langchain langgraph langchain-community langchain-text-splitters

print("LangChain Integration 패키지 설치 중...")
%pip install -qU langchain-openai langchain-qdrant

print("문서 파싱 라이브러리 설치 중...")
%pip install -qU pymupdf4llm docling paddleocr llama-parse

print("Semantic Chunking 라이브러리 설치 중...")
%pip install -qU "chonkie[all]"

print("그 외 유틸리티 설치 중...")
%pip install -qU python-dotenv ruff mypy ipywidgets

Python 버전: 3.13.9
 Python 버전 OK
LangChain V1.0 설치 중...
Note: you may need to restart the kernel to use updated packages.
LangChain Integration 패키지 설치 중...
Note: you may need to restart the kernel to use updated packages.
문서 파싱 라이브러리 설치 중...
Note: you may need to restart the kernel to use updated packages.
Semantic Chunking 라이브러리 설치 중...
Note: you may need to restart the kernel to use updated packages.
그 외 유틸리티 설치 중...
Note: you may need to restart the kernel to use updated packages.

 패키지 설치 완료!


In [2]:
# ----------------------------------------------------------------------------
# OpenAI / OpenRouter 모델 초기화 헬퍼
# ----------------------------------------------------------------------------
import os
from typing import Literal

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)


def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

In [14]:
# ============================================================================
# Reasoning 모델 테스트
# ============================================================================
import time
from pprint import pprint

try:
    reasoning_llm = create_openrouter_llm(model="openai/gpt-5", temperature=0.1)

    print("모델 생성 완료:")
    print(f"  - 실제 호출 모델명: {reasoning_llm.model_name}")
    print(f"  - Temperature: {reasoning_llm.temperature}")
    print(f"  - Max Tokens: {getattr(reasoning_llm, 'max_tokens', 'N/A')}")

    test_prompt = """다음 논리 퍼즐을 풀어주세요:

세 명의 친구 A, B, C가 있습니다.
- A는 항상 진실을 말합니다.
- B는 항상 거짓을 말합니다.
- C는 때때로 진실을, 때때로 거짓을 말합니다.

한 명이 "나는 B가 아니다"라고 말했습니다.
이 말을 한 사람은 누구일까요? 단계별로 추론해주세요."""

    print(f"테스트 질문: {test_prompt}")

    start_time = time.time()
    response = reasoning_llm.invoke(test_prompt)
    elapsed_time = time.time() - start_time

    print(f"[SUCCESS] 응답 생성 완료 (소요 시간: {elapsed_time:.2f}초)")
    print("응답 내용:")
    print("-" * 80)
    print(response.content)
    print("-" * 80)

    if hasattr(response, "response_metadata"):
        print("응답 메타데이터:")
        pprint(response.response_metadata)

    print("[OK] Reasoning 모델 테스트 완료!")

except Exception as e:
    print(f"[ERROR] Reasoning 모델 테스트 실패: {type(e).__name__}")
    print(f"상세 메시지: {str(e)}")
    import traceback

    print(f"Traceback: {traceback.format_exc()}")

모델 생성 완료:
  - 실제 호출 모델명: openai/gpt-5
  - Temperature: 0.1
  - Max Tokens: None
테스트 질문: 다음 논리 퍼즐을 풀어주세요:

세 명의 친구 A, B, C가 있습니다.
- A는 항상 진실을 말합니다.
- B는 항상 거짓을 말합니다.
- C는 때때로 진실을, 때때로 거짓을 말합니다.

한 명이 "나는 B가 아니다"라고 말했습니다.
이 말을 한 사람은 누구일까요? 단계별로 추론해주세요.


2025-11-09 15:37:04,103 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


[SUCCESS] 응답 생성 완료 (소요 시간: 42.73초)
응답 내용:
--------------------------------------------------------------------------------
결론: 누구나(A, B, C) 될 수 있으므로, 주어진 정보만으로는 화자를 특정할 수 없습니다.

단계별 추론
1) A가 말했다고 가정:
- 문장: “나는 B가 아니다.”
- 사실 여부: A ≠ B 이므로 참.
- A는 항상 진실만 말하므로 모순 없음 → A 가능.

2) B가 말했다고 가정:
- 문장: “나는 B가 아니다.”
- 사실 여부: 화자가 B이므로 거짓.
- B는 항상 거짓만 말하므로 모순 없음 → B 가능.

3) C가 말했다고 가정:
- 문장: “나는 B가 아니다.”
- 사실 여부: C ≠ B 이므로 참.
- C는 때로 진실을 말할 수 있으므로 모순 없음 → C 가능.

정리: A가 말하면 참, B가 말하면 거짓, C가 말하면 참이 됩니다. 세 경우 모두 각자의 성향과 모순되지 않으므로, 화자를 하나로 정할 수 없습니다.
--------------------------------------------------------------------------------
응답 메타데이터:
{'finish_reason': 'stop',
 'id': 'gen-1762670223-ERfRzgLhEWlIm02S7laO',
 'logprobs': None,
 'model_name': 'openai/gpt-5',
 'model_provider': 'openai',
 'system_fingerprint': None,
 'token_usage': {'completion_tokens': 1836,
                 'completion_tokens_details': {'accepted_prediction_tokens': None,
                                               'audio_tokens':

In [3]:
# ============================================================================
# Non-Reasoning 모델 테스트
# ============================================================================
try:
    non_reasoning_llm = create_openrouter_llm(
        model="openai/gpt-4.1-mini",
        temperature=0.1,
    )

    print("모델 생성 완료:")
    print(f"  - 실제 호출 모델명: {non_reasoning_llm.model_name}")
    print(f"  - Temperature: {non_reasoning_llm.temperature}")

    sample_question = (
        "LLM Input Context 를 RAG 기술로 가져왔을 때, 청킹 전략을 한 문장으로 요약해줘"
    )
    print(f"샘플 질문: {sample_question}")
    response = non_reasoning_llm.invoke(sample_question)
    print("응답:")
    print(response.content)

except Exception as e:
    print(f"[ERROR] Non-Reasoning 모델 테스트 실패: {type(e).__name__}")
    print(f"상세 메시지: {str(e)}")

모델 생성 완료:
  - 실제 호출 모델명: openai/gpt-4.1-mini
  - Temperature: 0.1
샘플 질문: LLM Input Context 를 RAG 기술로 가져왔을 때, 청킹 전략을 한 문장으로 요약해줘
응답:
청킹 전략은 긴 정보를 의미 단위로 나누어 LLM의 입력 길이 제한 내에서 효율적으로 처리하도록 하는 방법입니다.


In [3]:
# ============================================================================
# Embedding 모델 테스트 1 - OpenAI Embedding
# ============================================================================
try:
    embeddings = create_embedding_model("openai/text-embedding-3-small")

    result = embeddings.embed_query("Hello, world!")
    print(f"임베딩 길이: {len(result)}")

except Exception as e:
    print(f"Error: {e}")

  0%|          | 0/1 [00:00<?, ?it/s]

임베딩 길이: 1536


In [4]:
# ============================================================================
# Embedding 모델 테스트 2 - Qwen Embedding
# ============================================================================
try:
    # NOTE: 함수가 다른 것에 주의하세요.
    result = create_embedding_model_direct(
        model="qwen/qwen3-embedding-0.6b",
        encoding_format="float",
        input_text="Hello, world!",
    )
    print(f"성공! 임베딩 길이: {len(result)}")
except Exception as e:
    print(f"오류: {e}")
    import traceback

    print(traceback.format_exc())

성공! 임베딩 길이: 1024


In [5]:
# 샘플 PDF 파일 경로 설정
from pathlib import Path

# Day2/law 디렉토리에 있는 근로기준법 PDF 사용
LAW_DIR = Path("./law")
SAMPLE_PDF = LAW_DIR / "근로기준법(법률)(제20520호)(20250223).pdf"

if SAMPLE_PDF.exists():
    print(f"[OK] 샘플 PDF 발견: {SAMPLE_PDF.name}")
    print(f"     파일 크기: {SAMPLE_PDF.stat().st_size / 1024:.2f} KB")
    print(f"     전체 경로: {SAMPLE_PDF.absolute()}")

[OK] 샘플 PDF 발견: 근로기준법(법률)(제20520호)(20250223).pdf
     파일 크기: 211.04 KB
     전체 경로: /Users/jhj/Desktop/sds_class/Day2/notebooks/law/근로기준법(법률)(제20520호)(20250223).pdf


# Pre-Processing Document - 1. 문서 파싱

## 파서 선택 전략 (2025-11 기준)

### 파서 비교 -> 굉장히 주관적인 경험에 기반합니다.

| 파서 | 유형 | 속도 | 품질 | 비용 | 배포 | 테이블 | OCR | 추천 사용 사례 |
|------|------|------|------|------|------|--------|-----|----------------|
| **PyMuPDF4LLM** | 오픈소스 | ***** | *** | 무료 | 온프레미스 | ** | ❌ | 베이스라인, 빠른 프로토타입 |
| **Upstage** | OnPrem,SaaS | **** | ***** | 온프렘 가능하나 비쌈 | 클라우드 | ***** |  | 금융/법률, 복잡한 테이블 |
| **LlamaParse** | SaaS | *** | ***** | SaaS | 클라우드 | ***** |  | Agentic 처리, 최고 품질 |
| **Docling** | 오픈소스 | *** | **** | 무료 | 온프레미스 | **** |  | 보안 문서, VLM 기반 |
| **PaddleOCR** | 오픈소스 | ** | *** | 무료 | 온프레미스 | ** |  | OCR 중심, 다국어 |

---

### 선택 기준

#### 1. 온프레미스
보안 요구사항이나 비용 절감이 필요한 경우:
- **추천**: PyMuPDF4LLM (CPU) → Docling (GPU 가능 시)
- **장점**: 무료, 데이터 외부 유출 없음
- **단점**: Self-hosting 필요, Docling은 GPU 권장

#### 2. 품질 최우선 (클라우드 가능한 경우)
복잡한 문서 구조와 테이블 처리가 중요한 경우:
- **추천**: Upstage Document AI (한국어 최적화) → LlamaParse (글로벌)
- **장점**: 최고 품질, 설치 불필요
- **단점**: API 비용, 데이터 외부 전송 가능 여부 확인 필요

### 라이선스 및 제한사항

- **PaddleOCR**: Apache 2.0 (상업 사용 가능), 단 `중국 기업 모델`로 일부 프로젝트에서 사용 제한 가능
- **Docling**: MIT License (상업 사용 가능)
- **PyMuPDF4LLM**: AGPL v3 (주의: 상업적 재배포 시 소스 공개 필요)
- **SaaS (Upstage/LlamaParse)**: 데이터가 외부 서버로 전송됨 (데이터 프라이버시 검토 필요)

## 실습: Baseline - PyMuPDF4LLM

PyMuPDF4LLM의 특징:
- LLM 친화적 마크다운 출력
- 테이블을 마크다운 테이블로 변환
- 빠른 속도

### 참고문서
- https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/
- https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/api.html#pymupdf4llm-api

In [18]:
# PyMuPDF4LLM으로 PDF 파싱 (고품질 BaseLine Parser 설정)
import json
import time
from pathlib import Path
from pprint import pprint

import pymupdf4llm


def convert_pymupdf_objects_to_serializable(obj):
    """
    PyMuPDF 객체(Rect 등)를 JSON 직렬화 가능한 형태로 재귀적으로 변환합니다.

    Args:
        obj: 변환할 객체 (dict, list, Rect 등)

    Returns:
        JSON 직렬화 가능한 객체
    """
    # Rect 객체 처리
    if hasattr(obj, "__class__") and obj.__class__.__name__ == "Rect":
        # Rect를 (x0, y0, x1, y1) 튜플로 변환
        return tuple(obj)

    # 딕셔너리 처리
    elif isinstance(obj, dict):
        return {key: convert_pymupdf_objects_to_serializable(value) for key, value in obj.items()}

    # 리스트 처리
    elif isinstance(obj, list):
        return [convert_pymupdf_objects_to_serializable(item) for item in obj]

    # 튜플 처리 (내부에 Rect가 있을 수 있음)
    elif isinstance(obj, tuple):
        return tuple(convert_pymupdf_objects_to_serializable(item) for item in obj)

    # 기타 타입은 그대로 반환
    else:
        return obj


try:
    if SAMPLE_PDF.exists():
        start = time.time()

        # 이미지 저장 경로 설정
        image_dir = Path("./images")
        image_dir.mkdir(exist_ok=True)

        # PDF를 마크다운으로 변환 (고품질 설정)
        pymupdf4llm_result = pymupdf4llm.to_markdown(
            str(SAMPLE_PDF),
            show_progress=True,
            # ==========================================
            # 이미지 관련 설정
            # ==========================================
            write_images=True,  # 이미지를 파일로 저장
            image_path=str(image_dir),  # 이미지 저장 경로
            image_format="png",  # PNG 포맷 사용
            image_size_limit=0.05,  # 최소 이미지 크기 (페이지의 5%)
            dpi=150,  # 이미지 해상도
            ignore_images=False,  # 이미지 포함
            # ==========================================
            # 텍스트 추출 품질 향상
            # ==========================================
            force_text=True,  # 이미지 배경이 있어도 텍스트 출력
            detect_bg_color=True,  # 배경색 감지로 정확도 향상
            ignore_alpha=False,  # 투명한 텍스트는 제외
            fontsize_limit=3,  # 최소 폰트 크기
            # ==========================================
            # 테이블 감지 설정
            # ==========================================
            table_strategy="lines_strict",  # 엄격한 테이블 감지 전략
            # ==========================================
            # 그래픽 및 코드 처리
            # ==========================================
            ignore_graphics=False,  # 벡터 그래픽 포함
            ignore_code=False,  # 코드 형식 유지
            # ==========================================
            # 페이지 및 레이아웃 설정
            # ==========================================
            page_chunks=True,  # 전체 문서 = JSON 으로 리턴
            page_separators=True,  # 페이지 구분자 사용
            margins=0,  # 여백 포함
            page_width=800,  # 페이지 너비 (default=612)
            # ==========================================
            # 추가 설정
            # ==========================================
            extract_words=False,  # 기본 텍스트 추출만 사용
            use_glyphs=False,  # Unicode 사용
            embed_images=False,  # base64 인코딩 사용 안함 (write_images와 배타적)
        )

        elapsed = time.time() - start

        print("\n [OK] 파싱 완료")
        print(f"     소요 시간: {elapsed:.2f}초")

        # 이미지 저장 확인
        saved_images = list(image_dir.glob("*.png"))
        print(f"     저장된 이미지: {len(saved_images)}개")

        print("\n텍스트:")
        print("-" * 80)
        pprint(pymupdf4llm_result)
        print("-" * 80)

        # NOTE: PyMuPDF 객체를 직렬화 가능한 형태로 변환
        serializable_result = convert_pymupdf_objects_to_serializable(pymupdf4llm_result)

        # 파일로 저장
        with open(f"./chunks/{SAMPLE_PDF.name.replace('.pdf', '')}_pymupdf4llm.json", "w") as f:
            json.dump(serializable_result, f, indent=4, ensure_ascii=False)

        print("\n [SAVED] JSON 파일 저장 완료")

    else:
        print("[SKIP] 샘플 PDF 파일이 없어 실습을 건너뜁니다.")
        markdown_text = ""

except ImportError:
    print("[NOT INSTALLED] pymupdf4llm이 설치되지 않았습니다.")
except Exception as e:
    print(f"오류 발생: {type(e).__name__}")
    print(f" 메시지: {str(e)}")
    import traceback

    print(traceback.format_exc())

Processing law/근로기준법(법률)(제20520호)(20250223).pdf...
[                                        ] (0/2[=                                       ] ( 1/2=[===                                     ] ( 2/2=[=====                                   ] ( 3/2=[=======                                 ] ( 4/2=[=========                               ] ( 5/2=[===========                             ] ( 6/2=[=============                           ] ( 7/2=[===============                         ] ( 8/2=[=================                       ] ( 9/2=[===================                     ] (10/2[====================                    ] (11/21=[======================                  ] (12/21=[========================                ] (13/21=[==========================              ] (14/21=[============================            ] (15/21=[==============================          ] (16/21=[================================        ] (17/21=[==================================      ] (18/21=[============

## 실습: Upstage Document Parser (국내 Document AI SaaS)

Upstage Document Parser의 특징:
- 고품질 테이블 추출 (금융/법률 등 복잡한 문서에 최적화)
  > JSON, HTML, Markdown 형식으로 출력 가능
- Layout Analysis + OCR 통합
- API 기반 SaaS (별도 설치 불필요)
- 유료 서비스

### API 키 발급 방법
1. https://console.upstage.ai 접속
2. 회원가입 및 로그인
3. Dashboard > API Keys 메뉴에서 키 생성
4. .env 파일에 `UPSTAGE_API_KEY=...` 추가

### Upstage Document Parse 모델 제약사항 및 상세 정보

**최신버전 모델 Alias:** `document-parse`

#### 주요 제약사항

1. **파일 형식 지원**
   - PDF (우선 지원)
   - 스캔된 이미지 (OCR 자동 적용)
   - 스프레드시트, 슬라이드 등 다양한 문서 형식

2. **파일 크기 및 페이지 제한**
   - 최대 파일 크기: 제한 있음 (공식 문서 확인 필요)
   - 대용량 문서 처리 시 timeout 설정 중요 (권장: 300초 이상)
   - 페이지당 평균 처리 시간: 약 0.6초
   - 100페이지를 1분 이내 처리 가능

3. **처리 품질**
   - 복잡한 테이블 인식: ***** (금융/법률 문서 최적화)
   - OCR 정확도: 자동 OCR 활성화 시 높은 정확도
   - 레이아웃 분석: 기존 모델 대비 5% 이상 향상된 정확도

4. **비용**
   - 페이지당 약 $0.01 (변동 가능, 공식 문서 확인)
   - 대량 문서 처리 시 예산 관리 필요
   - 무료 티어: 약 100페이지 (신규 가입 시)

#### API 사양

- **엔드포인트:** `https://api.upstage.ai/v1/document-ai/document-parse`
- **인증 방식:** Bearer Token (API 키)
- **요청 형식:** multipart/form-data
- **응답 형식:** JSON
  - `content.paragraphs[]`: 문단 정보 (text, page, id, section 등)
  - `content.tables[]`: 표 정보 (markdown, csv, page, rows, cols 등)
  - `content.figures[]`: 이미지/차트 정보

#### 보안 및 데이터 프라이버시

- **데이터 전송:** 문서가 Upstage 서버로 전송됨
- **보안 문서 처리:** 외부 유출이 불가능한 문서는 온프레미스 솔루션(Docling, PyMuPDF) 사용
- **데이터 보존:** Upstage의 데이터 보존 정책 확인 필요

#### 참고 문서

- 공식 문서: https://console.upstage.ai/docs/capabilities/digitize/document-parsing
- 모델 상세: https://console.upstage.ai/docs/models/document-parse
- API 키 발급: https://console.upstage.ai (Dashboard > API Keys)

In [17]:
# Upstage Document AI로 PDF 파싱
upstage_api_key = os.getenv("UPSTAGE_API_KEY")

try:
    import asyncio

    import nest_asyncio
    from httpx import AsyncClient

    nest_asyncio.apply()

    async def parse_with_upstage_async_v2(file_path, api_key):
        """HTTPX AsyncClient를 사용한 Upstage Document Parse API 호출 (올바른 응답 구조)

        실제 응답 구조:
        {
          "api": "2.0",
          "content": {"html": "...", "markdown": "...", "text": "..."},
          "elements": [
            {"category": "paragraph|table|heading1|...", "content": {...}, "page": 1, "id": 0}
          ]
        }
        """
        # Upstage Document Parse API 엔드포인트
        url = "https://api.upstage.ai/v1/document-ai/document-parse"

        headers = {"Authorization": f"Bearer {api_key}"}

        async with AsyncClient(timeout=300.0) as client:
            with open(file_path, "rb") as f:
                files = {"document": (file_path.name, f, "application/pdf")}
                data = {
                    "model": "document-parse",
                    "ocr": "auto",
                    "output_formats": ["markdown", "html"],  # Markdown과 HTML 출력 요청
                }

                response = await client.post(url, headers=headers, files=files, data=data)

        response.raise_for_status()
        return response.json()

    if SAMPLE_PDF.exists():
        print("\nUpstage Document AI 파싱 (올바른 응답 구조)")
        print(f"   파일: {SAMPLE_PDF.name}")

        if upstage_api_key:
            print(f" API 키: {upstage_api_key[:10]}...")
            start = time.time()

            # 비동기 함수 실행
            result = asyncio.run(parse_with_upstage_async_v2(SAMPLE_PDF, upstage_api_key))

            elapsed = time.time() - start

            print(" 파싱 완료")
            print(f"   소요 시간: {elapsed:.2f}초")

            # content 확인
            content = result.get("content", {})
            print("\nContent:")
            print(f"   - HTML 길이: {len(content.get('html', ''))} 문자")
            print(f"   - Markdown 길이: {len(content.get('markdown', ''))} 문자")
            print(f"   - Text 길이: {len(content.get('text', ''))} 문자")

            # elements 확인
            elements = result.get("elements", [])
            print(f"\nElements: 총 {len(elements)}개")

            # 카테고리별 집계
            category_count = {}
            for elem in elements:
                cat = elem.get("category", "unknown")
                category_count[cat] = category_count.get(cat, 0) + 1

            print("\n  카테고리별 집계:")
            for cat, count in sorted(category_count.items(), key=lambda x: -x[1]):
                print(f"     - {cat}: {count}개")

            # 전체 Markdown 출력 (샘플)
            full_markdown = content.get("markdown", "")

            # TODO: 파일로 저장
            with open(f"./chunks/{SAMPLE_PDF.name.replace('.pdf', '')}_upstage.md", "w") as f:
                f.write(full_markdown)

            print("\n [SAVED] MD 파일 저장 완료")

            if full_markdown:
                print("\n전체 문서 Markdown:")
                print("-" * 80)
                pprint(full_markdown)
                print("-" * 80)
        else:
            print("UPSTAGE_API_KEY가 설정되지 않았습니다.")
    else:
        print("[SKIP] 샘플 PDF 파일이 없어 실습을 건너뜁니다.")

except ImportError as ie:
    print(f"[NOT INSTALLED] 필요한 라이브러리가 설치되지 않았습니다: {ie}")
    print("   설치: pip install httpx")
except Exception as e:
    print(f"오류 발생: {type(e).__name__}")
    print(f"   메시지: {str(e)}")
    import traceback

    print(f"\nTraceback:\n{traceback.format_exc()}")


Upstage Document AI 파싱 (올바른 응답 구조)
   파일: 근로기준법(법률)(제20520호)(20250223).pdf
 API 키: up_lSPIz0r...


2025-11-10 16:57:55,670 - INFO - HTTP Request: POST https://api.upstage.ai/v1/document-ai/document-parse "HTTP/1.1 200 OK"


 파싱 완료
   소요 시간: 6.61초

Content:
   - HTML 길이: 0 문자
   - Markdown 길이: 38722 문자
   - Text 길이: 0 문자

Elements: 총 362개

  카테고리별 집계:
     - paragraph: 216개
     - footer: 63개
     - list: 41개
     - header: 21개
     - heading1: 21개

전체 문서 Markdown:
--------------------------------------------------------------------------------
('근로기준법\n'
 '\n'
 '근로기준법\n'
 '\n'
 '[시행 2025. 2. 23.] [법률 제20520호, 2024. 10. 22., 일부개정]\n'
 '\n'
 '고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534\n'
 '고용노동부 (근로기준정책과 - 소년) 044-202-7535\n'
 '고용노동부 (근로기준정책과 - 임금) 044-202-7548\n'
 '고용노동부 (여성고용정책과 - 여성) 044-202-7475\n'
 '고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545\n'
 '고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973\n'
 '고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530\n'
 '고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549\n'
 '\n'
 '제1장 총칙\n'
 '\n'
 '제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는\n'
 '국민경제의 발전을 꾀하는 것을 목적으로 한다.\n'
 '\n'
 '제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. '
 '26.

## 실습: LlamaParse (Agentic VLM Document Parser)

LlamaParse의 특징:
- LlamaCloud(LlamaIndex)에서 제공하는 문서 파싱 서비스
- Agentic VLM 기반으로 복잡한 문서 구조 이해
- 테이블, 이미지, 수식 등 고급 요소 처리
- API 기반 SaaS (유료)

### API 키 발급 방법
1. https://cloud.llamaindex.ai 접속
2. 회원가입 및 로그인
3. API Keys 메뉴에서 키 생성
4. .env 파일에 `LLAMA_CLOUD_API_KEY=...` 추가

### 참고 문서
- Getting Started: https://developers.llamaindex.ai/python/cloud/llamaparse/getting_started/
- Modes and Presets: https://developers.llamaindex.ai/python/cloud/llamaparse/presets_and_modes/presets/
- Parsing Options: https://developers.llamaindex.ai/python/cloud/llamaparse/features/parsing_options/
- Multimodal: https://developers.llamaindex.ai/python/cloud/llamaparse/features/multimodal/
- Layout Extraction: https://developers.llamaindex.ai/python/cloud/llamaparse/features/layout_extraction/
- Metadata: https://developers.llamaindex.ai/python/cloud/llamaparse/features/metadata/
- Cache Options: https://developers.llamaindex.ai/python/cloud/llamaparse/features/cache_options/


In [19]:
# ============================================================================
# LlamaParse 고품질 설정 및 로컬 캐시
# ============================================================================

# ──────────────────────────────────────────────────────────────────────────────
# 1. 환경 확인
# ──────────────────────────────────────────────────────────────────────────────
import os

from dotenv import load_dotenv

load_dotenv()
llama_api_key = os.getenv("LLAMA_CLOUD_API_KEY")

if llama_api_key:
    print(f" LlamaParse API 키: {llama_api_key[:10]}...")
else:
    print("  LLAMA_CLOUD_API_KEY가 설정되지 않았습니다.")
    print("   https://cloud.llamaindex.ai 에서 API 키 발급 후 .env에 추가하세요.")

# ──────────────────────────────────────────────────────────────────────────────
# 2. 파싱 모드 및 모델 구성
# ──────────────────────────────────────────────────────────────────────────────
# 공식 문서 기반 모드 정의
MODE_CHOICES = {
    "cost_effective": {
        "parse_mode": "parse_page_with_llm",
        "model": None,  # 기본 모델 사용
        "description": "비용 효율적, 표준 문서, 빠른 속도",
    },
    "agentic": {
        "parse_mode": "parse_page_with_agent",
        "model": "openai-gpt-4-1-mini",
        "description": "고품질, 다이어그램/이미지 포함, 가성비 우수",
    },
    "agentic_plus_doc": {
        "parse_mode": "parse_page_with_agent",
        "model": "anthropic-sonnet-4.0",
        "description": "최고 품질(공식 문서 예시), 복잡한 레이아웃",
    },
}

# 기본값: Agentic Plus with Sonnet 4.5
DEFAULT_CONFIG = {
    "parse_mode": "parse_page_with_agent",
    "model": "anthropic-sonnet-4.5",
    "description": "최고 품질(커스텀), Claude Sonnet 4.5, 복잡한 문서",
}

# 고품질 옵션 (공식 문서 권장)
QUALITY_OPTIONS = {
    "high_res_ocr": True,  # 고해상도 OCR (느리지만 정확)
    "adaptive_long_table": True,  # 긴 표 자동 감지/적응
    "outlined_table_extraction": True,  # 테두리 있는 표 추출
    "output_tables_as_HTML": True,  # 표를 HTML로 출력 (마크다운 내)
}


# ──────────────────────────────────────────────────────────────────────────────
# 3. LlamaParse 파싱 실행 함수
# ──────────────────────────────────────────────────────────────────────────────
try:
    from llama_parse import LlamaParse

    def run_llamaparse(
        pdf_path: Path,
        config: dict,
        preset: str | None = None,
        api_key: str | None = None,
    ) -> tuple[str, dict]:
        """LlamaParse로 PDF 파싱 (설정 적용)

        Args:
            pdf_path: PDF 파일 경로
            config: 파싱 모드/모델 설정 (MODE_CHOICES 또는 DEFAULT_CONFIG)
            preset: Use-Case Preset (선택 사항, preset 지정 시 config 무시)
            api_key: LlamaParse API 키 (None이면 환경변수 사용)

        Returns:
            (마크다운 텍스트, 메타데이터 dict)

        Raises:
            RuntimeError: API 키 없거나 파싱 실패 시
        """
        if not api_key:
            api_key = os.getenv("LLAMA_CLOUD_API_KEY")
        if not api_key:
            raise RuntimeError("LLAMA_CLOUD_API_KEY가 필요합니다.")

        # 공통 옵션
        common_opts = {
            "api_key": api_key,
            "result_type": "markdown",
            "language": "ko",
            "verbose": True,
            **QUALITY_OPTIONS,
        }

        # config에서 parse_mode, model 추출
        parse_mode = config.get("parse_mode")
        model = config.get("model")

        if not parse_mode:
            raise ValueError("config에 parse_mode가 없습니다.")

        kwargs = {**common_opts, "parse_mode": parse_mode}
        if model:
            kwargs["model"] = model

        parser = LlamaParse(**kwargs)

        # 파싱 실행
        documents = parser.load_data(str(pdf_path))

        if not documents:
            raise RuntimeError("파싱 결과가 비어있습니다.")

        doc = documents[0]
        markdown_text = doc.text
        metadata = getattr(doc, "metadata", {})

        return markdown_text, metadata

    print(" LlamaParse 파싱 함수 정의 완료: run_llamaparse()")

except ImportError:
    print("   설치: pip install llama-parse")

 LlamaParse API 키: llx-M4XWre...
 LlamaParse 파싱 함수 정의 완료: run_llamaparse()


### 실습: Llama Parser 의 사용해보기

```python
# Cost-effective 모드
SELECTED_CONFIG = MODE_CHOICES["cost_effective"]

# Agentic 모드 (GPT-4.1-mini)
SELECTED_CONFIG = MODE_CHOICES["agentic"]

# Agentic Plus (Sonnet 4.0, 공식 문서 예시)
SELECTED_CONFIG = MODE_CHOICES["agentic_plus_doc"]

# 또는 DEFAULT (Sonnet 4.5, 현재 기본값)
SELECTED_CONFIG = DEFAULT_CONFIG
```

### 비교 포인트

- **속도**: Cost-effective가 가장 빠름
- **정확도**: Agentic Plus가 최고 (특히 복잡한 표, 차트)
- **비용**: Cost-effective < Agentic < Agentic Plus
- **캐시**: 동일 설정이면 재실행 시 즉시 로드

### LlamaParse 활용 방법

1. **프로토타입 단계**: Cost-effective로 빠르게 테스트
2. **프로덕션 단계**: 문서 유형에 따라 Agentic / 모델 바꿔가며 적합한 사양 테스트
3. **크리티컬 문서**: Agentic Plus(최고 정확도)
4. **캐시 활용**: 동일 문서 재파싱 시 API 비용 절감 가능.


In [9]:
# ============================================================================
# 실습: 다른 모드 테스트
# ============================================================================

# "cost_effective", "agentic_plus_doc", DEFAULT_CONFIG
SELECTED_CONFIG = MODE_CHOICES["agentic"]

try:
    print(f"\n파일: {SAMPLE_PDF.name}")
    print(f"   - parse_mode: {SELECTED_CONFIG.get('parse_mode')}")
    print(f"   - model: {SELECTED_CONFIG.get('model', '(default)')}")

    start_time = time.time()

    # 파싱 실행
    result_text, result_meta = run_llamaparse(
        pdf_path=SAMPLE_PDF,
        config=SELECTED_CONFIG,
        api_key=llama_api_key,
    )

    elapsed = time.time() - start_time
    print(f"\n 파싱 완료! (소요 시간: {elapsed:.2f}초)")

    # 결과 출력
    print("\n" + "=" * 80)
    print("파싱 결과")
    print("=" * 80)
    print(f"텍스트 길이: {len(result_text):,} 문자")
    print("\n처음 500자:")
    print("-" * 80)
    print(result_text[:500])
    print("...")
    print("-" * 80)

    # 통계
    lines = result_text.split("\n")
    tables = result_text.count("<table>")
    headings = len([line for line in lines if line.strip().startswith("#")])

    print("\n통계:")
    print(f"   - 줄 수: {len(lines):,}")
    print(f"   - HTML 표: {tables}")
    print(f"   - 헤딩: {headings}")

except Exception as e:
    print(f"\n오류: {type(e).__name__}")
    print(f"   메시지: {str(e)}")
    import traceback

    print("\nTraceback:")
    print(traceback.format_exc())


파일: 근로기준법(법률)(제20520호)(20250223).pdf
   - parse_mode: parse_page_with_agent
   - model: openai-gpt-4-1-mini
Started parsing the file under job_id 1ed83b8e-a6b6-4bf3-95e5-9723b9681c72

 파싱 완료! (소요 시간: 14.75초)

파싱 결과
텍스트 길이: 1,765 문자

처음 500자:
--------------------------------------------------------------------------------

# 근로기준법

[시행 2025. 2. 23.] [법률 제20520호, 2024. 10. 22., 일부개정]  
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534  
고용노동부 (근로기준정책과 - 소년) 044-202-7535  
고용노동부 (근로기준정책과 - 임금) 044-202-7548  
고용노동부 (여성고용정책과 - 여성) 044-202-7475  
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545  
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973  
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530  
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549  

## 제1장 총칙

### 제1조(목적)  
이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는 국민경제의 발전을 꾀하는 것을 
...
--------------------------------------------------------------------------------

통계:
   - 줄 수: 48
   - HTML 표: 0
   - 헤딩: 9


## 실습: Docling (오픈소스, VLM 기반)

Docling의 특징:
- IBM Research에서 개발한 MIT 라이센스의 오픈소스
- Layout Analysis + TableFormer (테이블 인식을 지원하는 VLM)
- PDF, DOCX, PPTX, 이미지 등 다양한 형식 지원
- 온프레미스 배포 가능

### 고급 기능
1. **OCR 엔진 선택**: Tesseract, EasyOCR, RapidOCR 등 각종 OCR 모델 지원 -> VLM 이 좋아지며 크게 장점은 없어짐
2. **원격 VLM 연동**: LM Studio, Ollama, Transformers, vLLM 등 원격 VLM 모델 활용 가능
3. **멀티모달 내보내기**: 텍스트+이미지+메타데이터를 Parquet로 내보내기 등
4. **파이프라인 옵션**: 페이지 제한, 테이블 추출 품질 조정, GPU 가속 등

공식 문서:
- 설치: https://docling-project.github.io/docling/installation/
- 고급 옵션: https://docling-project.github.io/docling/usage/advanced_options/
- VLM API 로 사용하기: https://docling-project.github.io/docling/examples/vlm_pipeline_api_model/
- 멀티모달 내보내기: https://docling-project.github.io/docling/examples/export_multimodal/
- Docling 사용 예제: https://docling-project.github.io/docling/examples/ 

In [21]:
# ============================================================================
# Docling으로 PDF 파싱
# ============================================================================

# ──────────────────────────────────────────────────────────────────────────────
# 설정 토글: 필요에 따라 True/False로 변경
# ──────────────────────────────────────────────────────────────────────────────
ENABLE_VLM = False  # 원격 VLM 사용 여부 (고품질 처리 필요 시 True)
VLM_PROVIDER = "openai_compatible"
VLM_HOST = "remote_host:12345"
VLM_MODEL = ""  # 사용할 VLM 모델
VLM_PROMPT = "Convert this page to OCR text."
ENABLE_MULTIMODAL_EXPORT = False  # 멀티모달 내보내기 (이미지 + 메타데이터 포함)
MAX_PAGES = None  # 페이지 제한 (None=전체, 숫자=해당 페이지까지)

print("\nDocling 설정:")
print(f"  원격 VLM: {ENABLE_VLM} (제공자: {VLM_PROVIDER if ENABLE_VLM else 'N/A'})")
print(f"  멀티모달 내보내기: {ENABLE_MULTIMODAL_EXPORT}")
print(f"  페이지 제한: {MAX_PAGES if MAX_PAGES else '전체'}")

try:
    # ──────────────────────────────────────────────────────────────────────────
    # 1. Import 필수 모듈
    # ──────────────────────────────────────────────────────────────────────────
    from docling.datamodel.base_models import ConversionStatus, InputFormat
    from docling.datamodel.pipeline_options import PdfPipelineOptions
    from docling.document_converter import DocumentConverter, PdfFormatOption

    # VLM 옵션 import
    if ENABLE_VLM:
        from docling.datamodel.pipeline_options import VlmPipelineOptions
        from docling.datamodel.pipeline_options_vlm_model import ApiVlmOptions, ResponseFormat
        from docling.pipeline.vlm_pipeline import VlmPipeline

    print(f"\nDocling 파싱 시작: {SAMPLE_PDF.name}")
    start = time.time()

    # ──────────────────────────────────────────────────────────────────────────
    # 2. 파이프라인 옵션 구성
    # ──────────────────────────────────────────────────────────────────────────
    if ENABLE_VLM:
        # VLM 파이프라인 사용
        print("   VLM 파이프라인 구성 중...")
        pipeline_options = VlmPipelineOptions(enable_remote_services=True)

        # 공급자별 VLM 옵션 설정
        if VLM_PROVIDER == "openai_compatible":
            pipeline_options.vlm_options = ApiVlmOptions(
                url=f"http://{VLM_HOST}/v1/chat/completions",
                params=dict(model=VLM_MODEL, max_tokens=4096),
                prompt=VLM_PROMPT,
                timeout=90,
                scale=2.0,
                temperature=0.2,
                response_format=ResponseFormat.DOCTAGS,
            )
        elif VLM_PROVIDER == "local":
            pipeline_options.vlm_options = ApiVlmOptions(
                url=f"http://localhost:1234/v1/chat/completions",
                params=dict(model=VLM_MODEL, max_tokens=4096),
                prompt=VLM_PROMPT,
                timeout=90,
                scale=2.0,
                temperature=0.2,
                response_format=ResponseFormat.DOCTAGS,
            )

        # VLM 파이프라인으로 변환기 생성
        converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_options=pipeline_options,
                    pipeline_cls=VlmPipeline,
                )
            }
        )
    else:
        # 일반 파이프라인 사용
        pipeline_options = PdfPipelineOptions()

        # 이미지 생성 (멀티모달 내보내기용)
        if ENABLE_MULTIMODAL_EXPORT:
            pipeline_options.generate_page_images = True
            pipeline_options.generate_picture_images = True

        # 변환기 생성
        converter = DocumentConverter(
            format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
        )

    # ──────────────────────────────────────────────────────────────────────────
    # 3. PDF 변환 실행
    # ──────────────────────────────────────────────────────────────────────────
    result = converter.convert(str(SAMPLE_PDF))

    elapsed = time.time() - start

    # ──────────────────────────────────────────────────────────────────────────
    # 4. 변환 상태 확인
    # ──────────────────────────────────────────────────────────────────────────
    if result.status != ConversionStatus.SUCCESS:
        print("\n변환 실패!")
        print(f"   상태: {result.status}")
        if hasattr(result, "errors") and result.errors:
            print("   오류:")
            for err in result.errors:
                print(f"     - {err}")
        raise RuntimeError(f"Docling 변환 실패: {result.status}")

    print("\n 파싱 완료")
    print(f"   소요 시간: {elapsed:.2f}초")
    print(f"   변환 상태: {result.status}")

    # ──────────────────────────────────────────────────────────────────────────
    # 5. 마크다운 출력
    # ──────────────────────────────────────────────────────────────────────────
    markdown_output = result.document.export_to_markdown()

    print(f"\n마크다운 길이: {len(markdown_output)} 문자")
    print("\n마크다운 텍스트:")
    print("-" * 80)
    pprint(markdown_output)
    print("-" * 80)

    # ──────────────────────────────────────────────────────────────────────────
    # 6. 문서 구조 정보
    # ──────────────────────────────────────────────────────────────────────────
    print("\n문서 구조:")
    print(
        f"   페이지 수: {len(result.document.pages) if hasattr(result.document, 'pages') else 'N/A'}"
    )
    print(
        f"   테이블 수: {len(result.document.tables) if hasattr(result.document, 'tables') else 'N/A'}"
    )
    print(
        f"   그림 수: {len(result.document.pictures) if hasattr(result.document, 'pictures') else 'N/A'}"
    )

    # ──────────────────────────────────────────────────────────────────────────
    # 7. 메타데이터 저장 (JSON)
    # ──────────────────────────────────────────────────────────────────────────
    import json
    from pathlib import Path

    output_dir = Path("./chunks")
    output_dir.mkdir(parents=True, exist_ok=True)

    basename = SAMPLE_PDF.stem
    metadata = {
        "filename": SAMPLE_PDF.name,
        "pages": len(result.document.pages) if hasattr(result.document, "pages") else 0,
        "tables": (len(result.document.tables) if hasattr(result.document, "tables") else 0),
        "pictures": (len(result.document.pictures) if hasattr(result.document, "pictures") else 0),
        "conversion_time_seconds": elapsed,
        "vlm_used": ENABLE_VLM,
        "vlm_provider": VLM_PROVIDER if ENABLE_VLM else None,
        "multimodal_export": ENABLE_MULTIMODAL_EXPORT,
        "status": str(result.status),
    }

    # 마크다운 저장
    md_path = output_dir / f"{basename}_docling.md"
    md_path.write_text(markdown_output, encoding="utf-8")
    print(f"\n마크다운 저장: {md_path}")

    # 메타 JSON 저장
    meta_path = output_dir / f"{basename}_docling_meta.json"
    meta_path.write_text(json.dumps(metadata, indent=2, ensure_ascii=False), encoding="utf-8")
    print(f"메타데이터 저장: {meta_path}")

    # ──────────────────────────────────────────────────────────────────────────
    # 8. 멀티모달 내보내기 (선택)
    # ──────────────────────────────────────────────────────────────────────────
    if ENABLE_MULTIMODAL_EXPORT:
        print("\n멀티모달 내보내기 시작...")
        try:
            # export_to_dict로 멀티모달 데이터 추출
            doc_dict = result.document.export_to_dict()

            # Parquet 저장 (pandas 필요)
            import pandas as pd

            # 페이지별로 구조화
            multimodal_data = []
            for page_idx, page in enumerate(result.document.pages):
                page_data = {
                    "page_num": page_idx + 1,
                    "text": "",
                    "images": [],
                    "tables": [],
                }

                # 텍스트 추출
                if hasattr(page, "text"):
                    page_data["text"] = page.text

                # 이미지 정보
                if hasattr(result.document, "pictures"):
                    for pic in result.document.pictures:
                        if hasattr(pic, "page") and pic.page == page_idx + 1:
                            page_data["images"].append(
                                {
                                    "id": getattr(pic, "id", ""),
                                    "caption": getattr(pic, "caption", ""),
                                }
                            )

                # 테이블 정보
                if hasattr(result.document, "tables"):
                    for table in result.document.tables:
                        if hasattr(table, "page") and table.page == page_idx + 1:
                            page_data["tables"].append(
                                {
                                    "id": getattr(table, "id", ""),
                                    "caption": getattr(table, "caption", ""),
                                }
                            )

                multimodal_data.append(page_data)

            df = pd.DataFrame(multimodal_data)
            parquet_path = output_dir / f"{basename}_multimodal.parquet"
            df.to_parquet(parquet_path, index=False)
            print(f"   Parquet 저장: {parquet_path}")
            print(f"   데이터 형태: {df.shape}")

        except ImportError:
            print("   pandas가 설치되지 않아 Parquet 저장을 건너뜁니다.")
            print("     설치: pip install pandas pyarrow")
        except Exception as e_mm:
            print(f"  멀티모달 내보내기 오류: {e_mm}")

    # 이후 청킹에 사용할 변수 저장
    if "markdown_text" not in locals() or not markdown_text:
        markdown_text = markdown_output

except Exception as e:
    print(f"\n오류 발생: {type(e).__name__}")
    print(f"   메시지: {str(e)}")
    import traceback

    print(f"\nTraceback:\n{traceback.format_exc()}")

2025-11-10 17:02:55,966 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-10 17:02:55,968 - INFO - Going to convert document batch...
2025-11-10 17:02:55,968 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f9730ffaa6e7f8d4fb0c98c8df3f18cb
2025-11-10 17:02:55,968 - INFO - Auto OCR model selected ocrmac.
2025-11-10 17:02:55,969 - INFO - Accelerator device: 'mps'



Docling 설정:
  원격 VLM: False (제공자: N/A)
  멀티모달 내보내기: False
  페이지 제한: 전체

Docling 파싱 시작: 근로기준법(법률)(제20520호)(20250223).pdf


2025-11-10 17:02:57,409 - INFO - Accelerator device: 'mps'
2025-11-10 17:02:57,566 - INFO - Processing document 근로기준법(법률)(제20520호)(20250223).pdf
2025-11-10 17:03:00,302 - INFO - Finished converting document 근로기준법(법률)(제20520호)(20250223).pdf in 4.34 sec.



 파싱 완료
   소요 시간: 4.34초
   변환 상태: ConversionStatus.SUCCESS

마크다운 길이: 41431 문자

마크다운 텍스트:
--------------------------------------------------------------------------------
('## 제1장 총칙\n'
 '\n'
 '- 제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는 국민경제의 발전을 '
 '꾀하는 것을 목적으로 한다.\n'
 '\n'
 '제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. &lt;개정 2018. 3. 20., 2019. 1. 15., 2020. '
 '5. 26.&gt;\n'
 '\n'
 "1. '근로자'란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.\n"
 "2. '사용자'란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를 말한다.\n"
 "3. '근로'란 정신노동과 육체노동을 말한다.\n"
 "4. '근로계약'이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체 결된 계약을 말한다.\n"
 "5. '임금'이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을 말한다.\n"
 "6. '평균임금'이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을 그 기간의 총일수로 "
 '나눈 금액을 말한다. 근로자가 취업한 후 3개월 미만인 경우도 이에 준한다.\n'
 "7. '1주'란 휴일을 포함한 7일을 말한다.\n"
 "8. '소정(所定)근로시간'이란 제50조, 제69조 본문 또는 「산업안전보건법」 제139조제1항에 따른 근로시간의 범위에 서 근로자와 "
 '사용자 사이에 정한 근로시간을 말한다.\n'
 "9. '단

## 파서 비교 및 선택 가이드

### 5가지 파서 상세 비교

| 파서 | 속도 | 품질 | 비용 | 배포 | 테이블 | OCR | 추천 사용 사례 |
|------|------|------|------|------|--------|-----|----------------|
| **PyMuPDF4LLM** | ***** | *** | 무료 | 온프레미스 | ** | X | 단순 PDF, 베이스라인, 빠른 프로토타이핑 |
| **Upstage** | **** | ***** | $$$ | SaaS | ***** | O | 금융/법률 문서, 복잡한 테이블, 외부 유출 가능 |
| **LlamaParse** | *** | ***** | $$$$ | SaaS | ***** | O | 최고 품질 필요, 복잡한 레이아웃, Agentic 처리 |
| **Docling** | *** | **** | 무료 | 온프레미스 | **** | O | 온프레미스 필수, VLM 기반, 모델 변경 가능 |
| **PaddleOCR** | *** | **** | 무료 | 온프레미스 | **** | O | 온프레미스 필수, VLM 기반, `중국` 이슈 |

### 실무 권장사항

#### 1. 프로젝트 초기 (프로토타입)
- **PyMuPDF4LLM** 사용
- 이유: 빠른 구현, 무료, 설치 간단
- 한계: 복잡한 테이블이나 이미지 기반 문서는 제한적

#### 2. 프로덕션 (품질 중시)
**외부 유출 가능한 문서:**
- **Upstage Document AI** (금융/법률 등 국내 문서 다수)
  - 장점: 테이블 인식 우수, 한국어 최적화
  - 비용: API 호출당 과금, 대량 처리 시 비용 고려
  
- **LlamaParse** (최고 품질 필요)
  - 장점: Agentic VLM으로 문맥 이해, 복잡한 레이아웃
  - 비용: 가장 비쌈, 크리티컬한 문서에만 사용

**외부 유출 불가능한 문서 (온프레미스):**
- **Docling 으로 Pipeline 구성을 추천** 
  - 장점: VLM 기반, 테이블/이미지 처리, 무료
  - 요구사항: GPU 필요 (빠른 처리 위해)
  - 배포: Docker 컨테이너로 자체 호스팅


### 주의사항

#### PaddleOCR, DeepSeek OCR, Qwen3-VL 등 중국산 VLM 모델 사용 시
- **국가적 이슈**: 프로젝트에서 중국산 모델 사용 제한하는 경우
- **대안**: OpenSource VLM 또는 국산 VLM 사용(국산 VLM 중 상용으로 사용 가능한 Open Source 모델은 없음)
  > 대부분 SaaS 이거나, 상용으로 판매하는 제품만 있습니다.  
  > NCSOFT/VARCO-VISION-2.0-1.7B-OCR(CC-BY-NC 라이센스), https://www.koreadeep.com/deep-ocr(한국딥러닝 이라는 회사)

#### SaaS 파서 사용 시
- **데이터 보안**: 민감한 문서는 온프레미스 솔루션 사용
- **비용 관리**: 문서당 비용 계산, 월 예산 설정
- **API 제한**: Rate limit, 동시 요청 수 확인


---

## OCR 및 Parse 단계 이후 고려 사항

1. **청킹 전략**: 파싱된 마크다운을 적절한 청크로 분할
   - RecursiveCharacterTextSplitter (LangChain 기본)
   - SemanticChunker (의미 기반)

2. **임베딩 및 인덱싱**: VectorDB(Qdrant 등)에 저장
   - OpenAI Embeddings 또는 On-Premise Embedding
   - 메타데이터 필터링 활용

3. **RAG 파이프라인 구축**: LangGraph
   - Retrieval 최적화
   - 프롬프트 엔지니어링 + 컨텍스트 엔지니어링

In [22]:
# TODO: 나머지 문서를 다 파싱해봐주세요.